In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime
from sklearn import datasets, ensemble

from evidently.dashboard import Dashboard
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.dashboard.tabs import DataDriftTab, NumTargetDriftTab, RegressionPerformanceTab

from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles

C:\Users\diego\anaconda3\envs\curso\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
C:\Users\diego\anaconda3\envs\curso\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


In [2]:
content = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), 
                            header=0, 
                            sep=',', 
                            parse_dates=['dteday'], 
                            index_col='dteday')


In [3]:
raw_data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [4]:
# Regression training
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [5]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [8]:
reference.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [9]:
current.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-29,619,1,0,1,0,0,6,0,1,0.22,0.1970,0.64,0.3582,2,26,28
2011-01-29,620,1,0,1,1,0,6,0,1,0.22,0.2273,0.64,0.1940,0,20,20
2011-01-29,621,1,0,1,2,0,6,0,1,0.22,0.2273,0.64,0.1642,0,15,15
2011-01-29,622,1,0,1,3,0,6,0,1,0.20,0.2121,0.64,0.1343,3,5,8
2011-01-29,623,1,0,1,4,0,6,0,1,0.16,0.1818,0.69,0.1045,1,2,3


In [10]:
regressor = ensemble.RandomForestRegressor(random_state = 0, 
                                            n_estimators = 50)

regressor.fit(reference[numerical_features + categorical_features], 
            reference[target])

RandomForestRegressor(n_estimators=50, random_state=0)

In [11]:
ref_prediction = regressor.predict(reference[numerical_features + categorical_features])


In [13]:
current_prediction = regressor.predict(current[numerical_features + categorical_features])

In [14]:
reference['prediction'] = ref_prediction
current['prediction'] = current_prediction

In [16]:
reference.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,prediction
dteday,,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,23.24
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,34.62
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,26.66
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,11.66
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,2.70


In [17]:
current.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,prediction
dteday,,,,,,,,,,,,,,,,,
2011-01-29,619,1,0,1,0,0,6,0,1,0.22,0.1970,0.64,0.3582,2,26,28,19.80
2011-01-29,620,1,0,1,1,0,6,0,1,0.22,0.2273,0.64,0.1940,0,20,20,17.98
2011-01-29,621,1,0,1,2,0,6,0,1,0.22,0.2273,0.64,0.1642,0,15,15,14.20
2011-01-29,622,1,0,1,3,0,6,0,1,0.20,0.2121,0.64,0.1343,3,5,8,6.58
2011-01-29,623,1,0,1,4,0,6,0,1,0.16,0.1818,0.69,0.1045,1,2,3,3.44


In [18]:
column_mapping = ColumnMapping()

column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [19]:
regression_perfomance_dashboard = Dashboard(tabs=[RegressionPerformanceTab()])
regression_perfomance_dashboard.calculate(reference, None, column_mapping=column_mapping)


In [23]:
regression_perfomance_dashboard.save("reports/index.html")

In [24]:
regression_perfomance_dashboard.calculate(reference, current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'], 
                                            column_mapping=column_mapping)

In [25]:
regression_perfomance_dashboard.save("data/regression_performance_after_week1.html")